# Rustworkx Graph Generator

In [1]:
%pip install -qqq rustworkx numpy pandas typing
%pip install -qqq torch
%pip install -qqq json


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
import json
import rustworkx as rx
from typing import Dict, List, Tuple
import numpy as np
#%pip install louvain
import louvain


device = "mps" if torch.backends.mps.is_available() else "cpu"

/var/folders/1b/rgchpsl92j197mwwwch1mbmw0000gn/T/ipykernel_77860/2654540550.py:7: DeprecationWarning: This package has been superseded by the `leidenalg` package and will no longer be maintained. Please upgrade to the `leidenalg` package.
  import louvain


# VERIS_MITRE LOADING

In [3]:
file_path = "/Users/nullzero/Documents/repos/opencti/kg_infosec/veris1_3_7-mappings-enterprise.json"

In [10]:
def load_mitre_data(file_path: json) -> Dict:
    with open(file_path, 'r') as f:
        data_dict = json.load(f)
        return data_dict

import rustworkx as rx
from typing import List, Dict, Tuple

def create_knowledge_graph(data: List[Dict]) -> Tuple[rx.PyGraph, Dict[str, int]]:
    """
    Creates a knowledge graph from MITRE ATT&CK data.

    Args:
        data (List[Dict]): The MITRE ATT&CK data, where each dictionary represents a technique, tactic, etc.

    Returns:
        Tuple[rx.PyGraph, Dict[str, int]]: A tuple containing the graph and a node map.
    """
    G = rx.PyGraph()
    
    node_map = {}

    for entry in data:

        technique_id = entry.get('technique_id')
        technique_name = entry.get('technique_name')
        tactic = entry.get('tactic')
        related_techniques = entry.get('related_techniques', [])
        intrusion_sets = entry.get('intrusion_sets', [])


        if technique_id not in node_map:
            node_index = G.add_node((technique_id, technique_name, tactic))
            node_map[technique_id] = node_index
        else:
            node_index = node_map[technique_id]
        

        for related_technique in related_techniques:
            related_id = related_technique.get('technique_id')
            related_name = related_technique.get('technique_name')
            if related_id not in node_map:
                related_index = G.add_node((related_id, related_name, tactic))
                node_map[related_id] = related_index
            else:
                related_index = node_map[related_id]
            
    
            G.add_edge(node_index, related_index, "related")
        

        for intrusion_set in intrusion_sets:
            set_id = intrusion_set.get('set_id')
            set_name = intrusion_set.get('set_name')
            if set_id not in node_map:
                set_index = G.add_node((set_id, set_name, "intrusion_set"))
                node_map[set_id] = set_index
            else:
                set_index = node_map[set_id]
            
            # Create an undirected edge between the technique and the intrusion set
            G.add_edge(node_index, set_index, "associated_with")
    
    return G, node_map



In [11]:
def analyze_graph(G: rx.PyGraph) -> Tuple[List[int], Dict, List[float], List[float]]:
    # Calculate node degrees
    degrees = G.degree()
    
    # Identify communities
    adj_matrix = rx.adjacency_matrix(G)
    partition = louvain.best_partition(adj_matrix)
    
    # Calculate clustering coefficients
    clustering_coeffs = rx.clustering_coefficients(G)
    
    # Calculate betweenness centrality
    betweenness = rx.betweenness_centrality(G)
    
    return degrees, partition, clustering_coeffs, betweenness

def identify_pivotal_nodes(G: rx.PyGraph, betweenness: List[float], top_n: int = 10) -> List[Tuple[int, float]]:
    return sorted(enumerate(betweenness), key=lambda x: x[1], reverse=True)[:top_n]


In [18]:
def create_knowledge_graph(file_path=file_path):
    data = load_mitre_data(file_path=file_path)
    G, node_map = create_knowledge_graph(data)
    return G, node map

def main(G):
    total_nodes = G.num_nodes()
    
    degrees = [G.degree(node) for node in node_map]
    communities, clustering_coeffs, betweenness = analyze_graph(G)
    
    # Identify pivotal nodes
    pivotal_nodes = identify_pivotal_nodes(G, betweenness)
    
    # Print summary statistics
    print(f"Number of nodes: {G.num_nodes()}")
    print(f"Number of edges: {G.num_edges()}")
    print(f"Average degree: {sum(degrees) / len(degrees):.2f}")
    print(f"Number of communities: {len(set(communities.values()))}")
    print(f"Average clustering coefficient: {sum(clustering_coeffs) / len(clustering_coeffs):.4f}")
    
    print("\nTop 10 pivotal nodes (highest betweenness centrality):")
    for node_id, centrality in pivotal_nodes:
        node_data = G.get_node_data(node_id)
        print(f"  {node_data[2]} ({node_data[0]}): {centrality:.4f}")

    return {"G": G, "data_results": {"degrees": degrees, "communities": communities, "clustering_coeffs": clustering_coeffs, "betweenness": betweenness}}
    # Here you could add more analysis, such as:
    # - Identify common attack patterns
    # - Find relationships between different intrusion sets
    # - Analyze the structure of specific sub-techniques

G, node_map = create_knowledge_graph(file_path=file_path)
main()

AttributeError: 'str' object has no attribute 'get'

In [ ]:
# Additional analyses
    top_attack_patterns = analyze_top_attack_patterns(G, node_map)
    hidden_relationships = analyze_hidden_relationships(G, node_map)
    df_analysis = prepare_df_analysis(G, node_map)
    
    # Print additional analyses
    print("\nTop 5 Attack Patterns:")
    for pattern in top_attack_patterns:
        print(f"  - {pattern}")

    print("\nHidden Relationships Between Intrusion Sets:")
    for relationship in hidden_relationships:
        print(f"  - {relationship}")

    print("\nDataFrame Analysis for A Priori Knowledge for Sampling:")
    print(df_analysis)

## Additional Analysis
# Here you could add more analysis, such as:
    # - Identify common attack patterns
    # - Find relationships between different intrusion sets
    # - Analyze the structure of specific sub-techniques


In [17]:
 # Calculate node degrees for all nodes
degrees = [G.degree(node) for node in G.node_indices()]

# Identify communities using connected components or any other community detection method
communities = {i: list(comp) for i, comp in enumerate(rx.connected_components(G))}

# Calculate clustering coefficients for all nodes
clustering_coeffs = list(rx.graph_all_clustering_coefficients(G).values())

# Calculate betweenness centrality for all nodes
betweenness = rx.graph_betweenness_centrality(G)

return degrees, communities, clustering_coeffs, betweenness

def identify_pivotal_nodes(G: rx.PyGraph, betweenness: Dict[int, float]) -> List[Tuple[int, float]]:
    """
    Identifies pivotal nodes in the graph based on betweenness centrality.

    Args:
        G (rx.PyGraph): The knowledge graph.
        betweenness (Dict[int, float]): Betweenness centrality values for nodes.

    Returns:
        List[Tuple[int, float]]: List of top 10 nodes with highest betweenness centrality.
    """
    # Sort nodes by their betweenness centrality to find pivotal nodes
    sorted_nodes = sorted(betweenness.items(), key=lambda x: x[1], reverse=True)
    # Return the top 10 pivotal nodes
    return sorted_nodes[:10]

def main(file_path: str):
    # Load MITRE ATT&CK data
    data = load_mitre_data(file_path=file_path)

    # Create the knowledge graph
    G, node_map = create_knowledge_graph(data)

    # Analyze the graph
    degrees, communities, clustering_coeffs, betweenness = analyze_graph(G)

    # Identify pivotal nodes
    pivotal_nodes = identify_pivotal_nodes(G, betweenness)

    # Print summary statistics
    print(f"Number of nodes: {G.num_nodes()}")
    print(f"Number of edges: {G.num_edges()}")
    print(f"Average degree: {sum(degrees) / len(degrees):.2f}")
    print(f"Number of communities: {len(communities)}")
    print(f"Average clustering coefficient: {sum(clustering_coeffs) / len(clustering_coeffs):.4f}")

    print("\nTop 10 pivotal nodes (highest betweenness centrality):")
    for node_id, centrality in pivotal_nodes:
        node_data = G[node_id]
        print(f"  Node {node_id}: {node_data} with centrality {centrality:.4f}")

    # Additional analyses
    top_attack_patterns = analyze_top_attack_patterns(G, node_map)
    hidden_relationships = analyze_hidden_relationships(G, node_map)
    df_analysis = prepare_df_analysis(G, node_map)

    # Print additional analyses
    print("\nTop 5 Attack Patterns:")
    for pattern in top_attack_patterns:
        print(f"  - {pattern}")

    print("\nHidden Relationships Between Intrusion Sets:")
    for relationship in hidden_relationships:
        print(f"  - {relationship}")

    print("\nDataFrame Analysis for A Priori Knowledge for Sampling:")
    print(df_analysis)

if __name__ == "__main__":
    main(file_path=file_path)

NameError: name 'G' is not defined